# Agentic RAG Playground by LlamaIndex

In [1]:
import datasets
from llama_index.core.schema import Document
from llama_index.core.tools import FunctionTool

from src.tools import GuestInfoRetrieverTool
from src.tools import DDGSearchTool
from src.tools import WeatherInfoTool
from src.tools import HubStatsTool

from src.retriever import query_guest_agent
from src.retriever import query_multi_step_agent

In [2]:
# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        text="\n".join([
            f"Name: {guest_dataset['name'][i]}",
            f"Relation: {guest_dataset['relation'][i]}",
            f"Description: {guest_dataset['description'][i]}",
            f"Email: {guest_dataset['email'][i]}"
        ]),
        metadata={"name": guest_dataset['name'][i]}
    )
    for i in range(len(guest_dataset))
]

In [3]:
# Initialize the tool
gitr = GuestInfoRetrieverTool(docs)

In [ ]:
# Example query
question = "Tell me about our guest named 'Lady Ada Lovelace'."

# Call the function using default parameters
response = await query_guest_agent(docs, question)

# Generalize the response handling (same as before)
if response and response.response:
    print("🎩 Alfred's Response:")
    print(response.response)
else:
    print("🎩 Alfred did not provide a response.")

In [ ]:
# Another example query
question = "Who is Charles Babbage?"

# Call the function with custom parameters
response = await query_guest_agent(
    docs,
    question,
    llm_model="phi3:mini", # Use a different LLM model
    system_prompt="You are a helpful AI assistant that provides information about people." # Use a different system prompt
)

# Generalize the response handling (same as before)
if response and response.response:
    print("🎩 Alfred's Response:")
    print(response.response)
else:
    print("🎩 Alfred did not provide a response.")

## Give Your Agent Access to the Web

In [4]:
# Create an instance of the DDGSearchTool class
ddst = DDGSearchTool()

In [5]:
# Use the search_tool method of the instance
response = ddst.search_tool("Who's the founder of Turkish Republic?")
print(response)

Founder and first president of the Turkish Republic. Although Atatürk ("Father of Turks") left this world behind in 1938, in Turkey his movie hero profile, icy-blue stare, elegant silhouette, and classically tailored suits remain everywhere to be seen. His portrait adorns the walls of teahouses as far flung as Van and Gaziantep.


In [6]:
# Initialize the tool
# The DuckDuckGoSearchTool class already initializes the FunctionTool internally.
# We just need to create an instance of the class.
wit = WeatherInfoTool()

In [7]:
hst = HubStatsTool()

In [8]:
# Example usage
response = hst.get_hub_stats("facebook")
print(response)

The most downloaded model by facebook is facebook/esmfold_v1 with 20,947,347 downloads.


## Integrate weather_info_tool and hub_stats_tool to Alfred

In [9]:
# Create a proper FunctionTool using the provided arguments

search_tool = FunctionTool.from_defaults(
        fn=ddst.search_tool,
        name="dd_search_tool",
        description=(
            "Use this tool to search the internet for general information about a topic, "
            "definitions, explanations, or factual details that are likely to be found on websites. "
            "This is a general-purpose search tool."
        )
    )

weather_info_tool = FunctionTool.from_defaults(
        fn=wit.get_weather_info,
        name="weather_info_tool",
        description="Use this tool ONLY when the user is asking for weather information for a specific location."
    )

hub_stats_tool = FunctionTool.from_defaults(
    fn=hst.get_hub_stats,
    name="hub_stats_tool",
    description=(
        "Use this tool *only* to find statistics about models on the Hugging Face Hub. "
        "Specifically, use this tool to find the most downloaded model for a given author or organization *registered on the Hugging Face Hub*. "
        "Input should be the exact author or organization name (e.g., 'facebook', 'google', 'microsoft'). "
        "Example queries this tool can answer: 'What is the most downloaded model by google on Hugging Face?', 'Tell me about the stats for microsoft on the Hub.'"
    )
)

guest_info_tool = FunctionTool.from_defaults(
    fn=gitr.get_guest_info,
    name="guest_info_tool",
    description="Retrieve comprehensive information about guests attending the gala by their name, including their relation, description, and contact details like email."
)

In [10]:
llm_model="llama3:latest"
question = "What is Facebook and what's their most popular model?"
tools_dict = {
     "dd_search_tool": search_tool,
     "hub_stats_tool": hub_stats_tool,
     "weather_info_tool": weather_info_tool
 }

# To print only the final answer:
response = await query_multi_step_agent(question, tools_dict, llm_model, print_details=False)
print("🎩 Alfred's Response:")
print(response)

Ollama server is running and ready.
Ollama server is already running.
Error: Could not find JSON block in LLM output.
LLM output: Here's the plan to answer the user query:

```
[
  {
    "task": "Search for general information about Facebook",
    "tool": "dd_search_tool",
    "tool_input": {"query": "What is Facebook"}
  },
  {
    "task": "Find Hugging Face author associated with Facebook's most popular model",
    "tool": "hub_stats_tool",
    "tool_input": {"author": "Facebook"} // Assuming Facebook has a Hugging Face author account
  }
]
```

In the first step, we use the `dd_search_tool` to search for general information about Facebook. This tool is suitable for finding general information on a topic.

In the second step, we assume that Facebook has a Hugging Face author account and uses the `hub_stats_tool` to find their most popular model. Since Facebook is not a typical machine learning model author, this assumption may not hold true in reality. If Facebook is indeed a Hugging

## Creating Your Gala Agent

In [12]:
query = "Tell me about Lady Ada Lovelace. What's her background?"
llm_model = "llama3:latest"
tools_dict = {
     "dd_search_tool": search_tool,
     "hub_stats_tool": hub_stats_tool,
     "weather_info_tool": weather_info_tool,
     "guest_info_tool": guest_info_tool
 }

# To print only the final answer:
response = await query_multi_step_agent(query, tools_dict, llm_model, print_details=False)
print("🎩 Alfred's Response:")
print(response)

Ollama server is running and ready.
Ollama server is already running.
🎩 Alfred's Response:
Lady Ada Lovelace was the daughter of Lord Byron, but her mother, Anne Isabella Milbanke, encouraged her interest in mathematics and science to counterbalance her father's artistic inclinations. She became fascinated with Charles Babbage's proposed analytic engine, seeing its potential for programming and calculation. Her education was largely self-directed, with guidance from Mary Somerville, a Scottish mathematician and astronomer. Lovelace's contributions to mathematics and computing lie in her notes on the analytic engine, where she described how it could be programmed and predicted its capabilities, making her often regarded as the first computer programmer.


In [13]:
query = "What's the weather like in Paris tonight? Will it be suitable for our fireworks display?"
llm_model = "gemma3:1b"
tools_dict = {
     "dd_search_tool": search_tool,
     "hub_stats_tool": hub_stats_tool,
     "weather_info_tool": weather_info_tool,
     "guest_info_tool": guest_info_tool
 }

# To print only the final answer:
response = await query_multi_step_agent(query, tools_dict, llm_model, print_details=False)
print("🎩 Alfred's Response:")
print(response)

Ollama server is running and ready.
Ollama server is already running.
🎩 Alfred's Response:
The weather in Paris tonight is rainy with a temperature of 15°C. It’s not suitable for your fireworks display.


In [15]:
query = "One of our guests is from Google. What can you tell me about their most popular model?"
llm_model = "gemma2:2b"
tools_dict = {
     "dd_search_tool": search_tool,
     "hub_stats_tool": hub_stats_tool,
     "weather_info_tool": weather_info_tool,
     "guest_info_tool": guest_info_tool
 }

# To print only the final answer:
response = await query_multi_step_agent(query, tools_dict, llm_model, print_details=False)
print("🎩 Alfred's Response:")
print(response)


Ollama server is running and ready.
Ollama server is already running.
🎩 Alfred's Response:
The most downloaded model by Google is google/electra-base-discriminator, with 17,910,835 downloads.  



In [11]:
query = "I need to speak with Dr. Nikola Tesla about recent advancements in wireless energy. Can you help me prepare for this conversation?"
llm_model = "llama3:latest"
tools_dict = {
     "dd_search_tool": search_tool,
     "hub_stats_tool": hub_stats_tool,
     "weather_info_tool": weather_info_tool,
     "guest_info_tool": guest_info_tool
 }

# To print only the final answer:
response = await query_multi_step_agent(query, tools_dict, llm_model, print_details=False)
print("🎩 Alfred's Response:")
print(response)


Ollama server is running and ready.
Ollama server is already running.
🎩 Alfred's Response:
To prepare for your conversation with Dr. Nikola Tesla about recent advancements in wireless energy, it's essential to familiarize yourself with the two primary methods: inductive wireless power transfer and magnetic resonant coupling WPT. Both methods utilize magnetic fields as the medium for energy transfer. In the case of magnetic resonant coupling WPT, studies initially didn't incorporate compensation networks to enhance system performance, relying solely on magnetic induction between two coils.
